In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import uuid
import os


In [3]:
# 웹 브라우저 열기
driver = webdriver.Chrome()
driver.maximize_window() 

In [4]:
# URL로 이동
base_url = "https://m.bunjang.co.kr/"
driver.get(base_url)

In [5]:
# 2. keyword 검색
input_box_css = "#root > div > div > div.sc-dfVpRl.fxYqEy > div.sc-gzOgki.ciqSeg > div.sc-iyvyFf.kPWKwK > div.sc-jKJlTe.keyjxL > div.sc-eNQAEJ.voMyM > input"
input_box = driver.find_element(By.CSS_SELECTOR, input_box_css)
input_box.send_keys(" ")
input_box.send_keys("유모차 스토케")
input_box.send_keys(Keys.ENTER)

# 모든 아이템 리스트

In [9]:
# 상품 href 리스트 만들기
item_card_css = "#root > div > div > div:nth-child(4) > div > div.sc-gacfCG.QBPXM > div > div > a"
item_card_list = driver.find_elements(By.CSS_SELECTOR, item_card_css)
len(item_card_list)

100

In [10]:
# 광고인지 체크
is_ad_css = "div.sc-gVyKpa.kXoKNq > span.sc-cpmKsF.buJhvy"
try:
    el = item_card_list[-15].find_element(By.CSS_SELECTOR, is_ad_css)
    is_ad = el.text == "광고"
except NoSuchElementException:
    is_ad = False

is_ad

True

In [11]:
# 판매 완료
is_completed_css = "div > div > div > img"
try:
    el = item_card_list[0].find_element(By.CSS_SELECTOR, is_completed_css)
    is_completed = el.get_attribute('alt') == "판매 완료"
except NoSuchElementException:
    is_completed = False

is_completed

False

In [38]:
from datetime import datetime, timedelta
import re

def parse_relative_time(text: str, now: datetime | None = None) -> datetime:
    """
    한국어 상대 시간을 실제 datetime으로 변환
    ex) "1분 전", "2시간 전", "3일 전", "3달 전"
    """
    if now is None:
        now = datetime.now()

    match = re.match(r"(\d+)(분|시간|일|달) 전", text.strip())
    if not match:
        raise ValueError(f"지원하지 않는 형식: {text}")

    value = int(match.group(1))
    unit = match.group(2)

    if unit == "분":
        return now - timedelta(minutes=value)
    elif unit == "시간":
        return now - timedelta(hours=value)
    elif unit == "일":
        return now - timedelta(days=value)
    elif unit == "달":
        # "달" → 30일로 가정
        return now - timedelta(days=value * 30)
    else:
        raise ValueError(f"알 수 없는 단위: {unit}")


# 사용 예시
now = datetime.now()
print(parse_relative_time("1분 전", now))   # → 2025-08-19 15:29:00
print(parse_relative_time("2시간 전", now)) # → 2025-08-19 13:30:00
print(parse_relative_time("3일 전", now))   # → 2025-08-16 15:30:00
print(parse_relative_time("3달 전", now))   # → 2025-05-21 15:30:00

2025-08-19 19:34:38.070541
2025-08-19 17:35:38.070541
2025-08-16 19:35:38.070541
2025-05-21 19:35:38.070541


In [40]:
# 업로드 날짜
# 1분 전, 2시간 전, 3일 전, 3달 전

uploaded_date_css = "div.sc-iGPElx.iPIFfd > div.sc-gtfDJT.hZQMHB > div.sc-fOICqy.bgZehm"
uploaded_date = item_card_list[0].find_element(By.CSS_SELECTOR, uploaded_date_css)
uploaded_date = parse_relative_time(uploaded_date.text)

# 아이템 상세 정보

In [43]:
driver.get(item_card_list[0].get_attribute("href"))

In [46]:
# 타이틀
title_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div.ProductSummarystyle__Basic-sc-oxz0oy-2.ifrXrN > div.ProductSummarystyle__Name-sc-oxz0oy-3.dZBHcg"
title = driver.find_element(By.CSS_SELECTOR, title_css).text

# 가격
price_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div.ProductSummarystyle__Basic-sc-oxz0oy-2.ifrXrN > div.ProductSummarystyle__PriceWrapper-sc-oxz0oy-4.dTIDFF > div"
price = driver.find_element(By.CSS_SELECTOR, price_css).text
price = re.sub("[^0-9]", "", price)

# 상품 상태
condition_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div.ProductSummarystyle__Value-sc-oxz0oy-21.eLyjky"
condition = driver.find_element(By.CSS_SELECTOR, condition_css).text

# 상품 정보
detail_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductBottom-sc-13cvfvh-14.fxuPQD > div.Productsstyle__ProductInfoContent-sc-13cvfvh-13.jzEavb > div > div.ProductInfostyle__Wrapper-sc-ql55c8-0.gPJVxW > div.ProductInfostyle__Description-sc-ql55c8-2.hWujk > div.ProductInfostyle__DescriptionContent-sc-ql55c8-3.eJCiaL > p"
detail = driver.find_element(By.CSS_SELECTOR, detail_css).text


In [16]:
import requests

def download_image(url: str, save_path: str, chunk_size: int = 1024) -> bool:
    """
    주어진 URL에서 이미지를 다운로드하여 지정한 경로에 저장합니다.

    Args:
        url (str): 이미지 URL
        save_path (str): 저장할 파일 경로
        chunk_size (int): 다운로드할 때 사용할 청크 크기 (기본 1024)

    Returns:
        bool: 성공 시 True, 실패 시 False
    """
    try:
        resp = requests.get(url, stream=True, timeout=10)
        if resp.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in resp.iter_content(chunk_size):
                    if chunk:  # keep-alive chunks 건너뛰기
                        f.write(chunk)
            print(f"다운로드 완료: {save_path}")
            return True
        else:
            print(f"다운로드 실패: HTTP {resp.status_code}")
            return False
    except Exception as e:
        print(f"오류 발생: {e}")
        return False


# 사용 예시
url = "https://media.bunjang.co.kr/product/345010693_1_1754091876_w480.jpg"
download_image(url, "bungae_image.jpg")


다운로드 완료: bungae_image.jpg


True

In [ ]:
# 이미지
image_list_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductImageWrapper-sc-13cvfvh-10.cXRuyi > div > div.sc-kLIISr.gWGEJy > div > img"
image_list = driver.find_elements(By.CSS_SELECTOR, image_list_css)

uid = uuid.uuid4()
image_path = "../../data/raw/" + str(uid)
os.makedirs(image_path, exist_ok=True)

for index, image in enumerate(image_list):
    url = image.get_attribute('src')
    download_image(url, f"{image_path}/bungaejangter_{uid}_{index}.jpg")

<selenium.webdriver.remote.webelement.WebElement (session="a8f516e1fb6f441351813f58befef09a", element="f.269483FADD768ECCF3A5984461F92AE0.d.93D58BD36743FE45D8B459E6C7485C94.e.214")>
다운로드 완료: ../../data/raw/affcfbbd-091e-490c-a3d3-39aa382b9979/bungaejangter_affcfbbd-091e-490c-a3d3-39aa382b9979_0.jpg
https://media.bunjang.co.kr/product/286798147_1_1724918065_w900.jpg
<selenium.webdriver.remote.webelement.WebElement (session="a8f516e1fb6f441351813f58befef09a", element="f.269483FADD768ECCF3A5984461F92AE0.d.93D58BD36743FE45D8B459E6C7485C94.e.229")>
다운로드 완료: ../../data/raw/affcfbbd-091e-490c-a3d3-39aa382b9979/bungaejangter_affcfbbd-091e-490c-a3d3-39aa382b9979_1.jpg
https://media.bunjang.co.kr/product/286798147_2_1724918065_w900.jpg
<selenium.webdriver.remote.webelement.WebElement (session="a8f516e1fb6f441351813f58befef09a", element="f.269483FADD768ECCF3A5984461F92AE0.d.93D58BD36743FE45D8B459E6C7485C94.e.230")>
다운로드 완료: ../../data/raw/affcfbbd-091e-490c-a3d3-39aa382b9979/bungaejangter_affcfbb

In [47]:
# 모든 정보 
item = {
    "title" : title,
    "detail" : detail,
    "condition" : condition,
    "uploaded_date" : uploaded_date,
    "is_completed" : is_completed,
    "price": price,
}
items = []
items.append(item)
print(item)

{'title': '최고급 명품 럭셔리 디럭스 유모차 스토케. 수도권 전철역 전지역 및 대전', 'detail': '최고급 명품 럭셔리 디럭스 유모차 스토케. 수도권 전철역 전지역 및 대전권 무료배달됩니다. 환불불가.', 'condition': '사용감 없음', 'uploaded_date': datetime.datetime(2025, 8, 19, 19, 31, 45, 129378), 'is_completed': False, 'price': '60000'}


In [49]:
df = pd.DataFrame(items)

In [50]:
df.to_csv("../../data/raw/bungaejangter.csv")